In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets , transforms
from torch.autograd import Variable
epoch = 10

In [47]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../mnist_data",
        train = True, download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                (0.1307,) , (0.3081,)
            )
        ])
    ),
    batch_size = 32 , shuffle = True,
    num_workers = 3 , pin_memory = True
)

In [48]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../mnist_data/",
        train = False , transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
                (0.1307,) , (0.3081,)
            )
        ])
    ),
    batch_size = 32 , shuffle = True,
    num_workers=1 , pin_memory=True
)

In [53]:
cuda = True
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(
            1 , 10, kernel_size=5
        ) #Ouput Size : (10,24,24)
        self.conv2 = nn.Conv2d(
            10,20 , kernel_size=5
        ) #output Size : (20,8,8)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self , x):
        x = F.relu(F.max_pool2d(
            self.conv1(x),2
        )) #Ouput Size : (10,12,12)
        x = F.relu(F.max_pool2d(
            self.conv2_drop(
                self.conv2(x)
            ),2
        )) #Output Size : (20,4,4)
        x = x.view(-1,320) #Flatten
        x = F.relu(self.fc1(x))
        x = F.dropout(x , training = self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x , dim=1)
model = Net()
model.cuda()

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [50]:
optimizer = optim.SGD(
    model.parameters() , lr = 0.05 , momentum = 0.5
)

In [51]:
def train(ep):
    model.train()
    for batch_idx , (data,target) in enumerate(train_loader):
        if cuda:
            data , target = data.cuda(),target.cuda()
        data,target = Variable(data) , Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output , target)
        loss.backward() #Calculate the Gradients
        optimizer.step() #Update the weights and biases
        if batch_idx%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                ep, batch_idx * len(data), len(train_loader.dataset),
100. * batch_idx / len(train_loader), loss.data[0]))
        

In [52]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data,target = data.cuda() , target.cuda()
        data,target = Variable(data , volatile = True) , Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output , target , size_average = False).data[0]
        pred = output.data.max(1 , keepdim = True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
100. * correct / len(test_loader.dataset)))

import time
t0 = time.time()
for ep in range(1, epoch + 1):
    train(ep)
    test()
print(time.time() - t0)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.347273
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.104676
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.753431
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.007271
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.314224
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.625260
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.518987
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.345156
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.310628
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.340154
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.737500
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.325984
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.212409
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.385725
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.205448
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.183332
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.429221
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.220435
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.052265

Test set: Average loss: 0.0995, Accuracy: 9703/10000 (97%)

Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.138637
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.087450
Train Epoch: 9 [19200/60000 (32%)]	Loss: 0.179604
Train Epoch: 9 [22400/60000 (37%)]	Loss: 0.154809
Train Epoch: 9 [25600/60000 (43%)]	Loss: 0.097153
Train Epoch: 9 [28800/60000 (48%)]	Loss: 0.010666
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.164515
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.152855
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.075827
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.184960
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.210132
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.176869
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.219696
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.408949
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.351329

Test set: Average loss: 0.0485, Accuracy: 9852/10000 (99%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.064164
Train Epoch: 10 [3200/60000 (5%)]	Loss: 0.311194
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.103051
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.1